In [1]:
# !pip install neo4j
import neo4j
import pandas as pd

In [2]:
def connect_db():
    driver = neo4j.GraphDatabase.driver(uri="neo4j://0.0.0.0:7687", auth=("neo4j","password"))
    session = driver.session(database="neo4j")
    return session
    
def wipe_out_db(session):
    # wipe out database by deleting all nodes and relationships
    
    # similar to SELECT * FROM graph_db in SQL
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

session = connect_db()
wipe_out_db(session) 

## Create Posts & Comments

In [4]:
query = '''
CREATE
  // Users
  (josh:User {userID: '1001', name: 'Josh', age: 23, location: 'New York', interests: ['music', 'movies']}),
  (sam:User {userID: '1002', name: 'Sam', age: 25, location: 'Paris', interests: ['books', 'travel']}),
  (riley:User {userID: '1003', name: 'Riley', age: 35, location: 'London', interests: ['sports', 'technology']}),
  (taylor:User {userID: '1004', name: 'Taylor', age: 28, location: 'San Francisco', interests: ['music', 'technology']}),
  (alex:User {userID: '1005', name: 'Alex', age: 30, location: 'Berlin', interests: ['cooking', 'travel']}),
  (morgan:User {userID: '1006', name: 'Morgan', age: 27, location: 'Sydney', interests: ['movies', 'books']}),

  // Posts
  (post1:Post {postID: 'post1', content: 'Music Festival Tips', timestamp: '2023-11-10T10:00:00', topic: 'music'}),
  (post2:Post {postID: 'post2', content: 'Top Travel Destinations', timestamp: '2023-11-10T11:00:00', topic: 'travel'}),
  (post3:Post {postID: 'post3', content: 'Latest Tech Trends', timestamp: '2023-11-10T12:00:00', topic: 'technology'}),
  (post4:Post {postID: 'post4', content: 'Book Review: Sci-Fi Adventures', timestamp: '2023-11-10T13:00:00', topic: 'books'}),
  (post5:Post {postID: 'post5', content: 'Exploring Movie Genres', timestamp: '2023-11-11T09:00:00', topic: 'movies'}),
  (post6:Post {postID: 'post6', content: 'Healthy Cooking Recipes', timestamp: '2023-11-11T10:00:00', topic: 'cooking'}),
  (post7:Post {postID: 'post7', content: 'Learning Photography Basics', timestamp: '2023-11-11T11:00:00', topic: 'photography'}),

  // Groups
  (groupMusic:Group {groupID: 'g1', name: 'Music Enthusiasts', description: 'A group for people who are passionate about music'}),
  (groupTravel:Group {groupID: 'g2', name: 'Travel Buffs', description: 'A group for people who love to travel and explore new places'}),
  (groupTech:Group {groupID: 'g3', name: 'Tech Innovators', description: 'A group for individuals interested in the latest technology trends'}),
  
  // Hashtag node type
  (hashtagMusic:Hashtag {name: '#MusicFestival'}),
  (hashtagTravel:Hashtag {name: '#TravelTips'}),
  (hashtagTech:Hashtag {name: '#TechNews'}),

  // User Post
  (josh)-[:POSTED]->(post1),
  (josh)-[:POSTED]->(post5),
  (sam)-[:POSTED]->(post2),
  (riley)-[:POSTED]->(post3),
  (morgan)-[:POSTED]->(post4),
  (alex)-[:POSTED]->(post6),
  
  // User comment
  (josh)-[:COMMENTED_ON {content: 'Great tips!'}]->(post2),
  (sam)-[:COMMENTED_ON {content: 'Interesting read.'}]->(post1),
  (riley)-[:COMMENTED_ON {content: 'I totally agree.'}]->(post3),
  (morgan)-[:COMMENTED_ON {content: 'Love this book too!'}]->(post4),
  (alex)-[:COMMENTED_ON {content: 'Nice recipes.'}]->(post6),
  (taylor)-[:COMMENTED_ON {content: 'Amazing photography skills!'}]->(post7),

  // Establishing relationships between Users and Groups
  (josh)-[:MEMBER_OF]->(groupMusic),
  (sam)-[:MEMBER_OF]->(groupTravel),
  (riley)-[:MEMBER_OF]->(groupTech),

  // Tagging Posts with Hashtags
  (post1)-[:TAGGED_WITH]->(hashtagMusic),
  (post2)-[:TAGGED_WITH]->(hashtagTravel),
  (post3)-[:TAGGED_WITH]->(hashtagTech),  

  // Likes
  (josh)-[:LIKED]->(post3),
  (sam)-[:LIKED]->(post4),
  (riley)-[:LIKED]->(post5),
  (taylor)-[:LIKED]->(post2),
  (alex)-[:LIKED]->(post1),
  (morgan)-[:LIKED]->(post3),

  // Friendships
  (josh)-[:FRIEND]->(sam),
  (josh)-[:FRIEND]->(taylor),
  (sam)-[:FRIEND]->(riley),
  (riley)-[:FRIEND]->(morgan),
  (morgan)-[:FRIEND]->(alex),
  (alex)-[:FRIEND]->(taylor)
'''
session.run(query)

In [5]:
def run_query_to_pandas(session, query):
    # run a query and return the results in a pandas dataframe
    
    result = session.run(query)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

def run_query_print_raw(session, query):
    result = session.run(query)
    
    for r in result:
        print(r.values())

## Queries

### 1.Retrieve a Property of a Specific User

In [6]:
query = '''
    MATCH (u:User {userID: '1001'}) 
    RETURN DISTINCT u.name, u.age, u.location, u.interests
'''

df = run_query_to_pandas(session, query)
display(df)

,u.name,u.age,u.location,u.interests
0,Josh,23,New York,"[music, movies]"


### 2. Find all Posts created by a specific User

In [7]:
query = '''
    MATCH (u:User)-[:POSTED]->(p:Post)
    WHERE u.userID = '1001'
    RETURN DISTINCT u.name,p.postID, p.content, p.timestamp, p.topic
'''

df = run_query_to_pandas(session, query)
display(df)

,u.name,p.postID,p.content,p.timestamp,p.topic
0,Josh,post5,Exploring Movie Genres,2023-11-11T09:00:00,movies
1,Josh,post1,Music Festival Tips,2023-11-10T10:00:00,music


### 3. Find all Users who posted a specific topic of Post (topic is a property of Post)

In [8]:
query = '''
    MATCH (u:User)-[:POSTED]->(p:Post)
    WHERE p.topic = 'technology'
    RETURN DISTINCT u.name
'''
df = run_query_to_pandas(session, query)
display(df)

,u.name
0,Riley


### 4.Find common interests between two specific Users. 

In [9]:
query = '''
    MATCH (u1:User), (u2:User) 
    WHERE u1.userID < u2.userID AND any(interest IN u1.interests WHERE interest IN u2.interests) 
    RETURN DISTINCT u1.name, u2.name, [i IN u1.interests WHERE i IN u2.interests] AS commonInterests
'''
df = run_query_to_pandas(session, query)
display(df)

,u1.name,u2.name,commonInterests
0,Josh,Taylor,[music]
1,Josh,Morgan,[movies]
2,Sam,Alex,[travel]
3,Sam,Morgan,[books]
4,Riley,Taylor,[technology]


### 5. Retrieve top 3 Users who created most Posts

In [10]:
query = '''
    MATCH (u:User)-[:POSTED]->(p:Post)
    RETURN u.name, COUNT(DISTINCT p) AS numPosts
    ORDER BY numPosts DESC
'''
df = run_query_to_pandas(session, query)
display(df)

,u.name,numPosts
0,Josh,2
1,Sam,1
2,Riley,1
3,Morgan,1
4,Alex,1


### 6. Retrieve Users who haven’t created any Posts

In [11]:
query = '''
    MATCH (u:User)
    WHERE NOT EXISTS ((u)-[:POSTED]->(:Post))
    RETURN u.name
'''
df = run_query_to_pandas(session, query)
display(df)

,u.name
0,Taylor


### 7. Given two Users, identify if they are indirectly connected through a chain of friends and, if so, return the connecting path. For instance, can you find a relationship path where Alice is friends with someone (say, Bob), who in turn is friends with Charlie?

In [12]:
query = '''
    MATCH path = (u1:User)-[:FRIEND*2..3]-(u3:User)
    WHERE u1 <> u3 AND length(path) > 1
    RETURN DISTINCT u1.name, u3.name
'''
df = run_query_to_pandas(session, query)
display(df)


,u1.name,u3.name
0,Josh,Riley
1,Josh,Morgan
2,Josh,Alex
3,Sam,Taylor
4,Sam,Alex
5,Sam,Morgan
6,Riley,Josh
7,Riley,Taylor
8,Riley,Alex
9,Taylor,Sam


### 8. Write a query to identify orphaned Users (Users who have no connections). Can you think of a scenario when this information is useful?

In [13]:
query = '''
    MATCH (u:User) 
    WHERE NOT (u)-[:FRIEND]-(:User) 
    RETURN u.name
'''
df = run_query_to_pandas(session, query)
display(df)

,u.name


In social media, identifying orphaned users — those without connections — is crucial for creating an engaging and inclusive community. Social platforms can use this data to strategically integrate these users into the community by analyzing their interests and suggesting relevant friends or groups. This personalized approach not only improves user satisfaction but also boosts the network's interactivity. By engaging these users, platforms can increase overall activity and foster a more dynamic social environment.